# Введение в искусственные нейронные сети
# Урок 5. Сверточные нейронные сети

## Практическое задание

<ol>
    <li>Попробовать улучшить точность распознования образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложить анализ с описанием того, что улучшает работу нейронной сети и что ухудшает.
    </li>
    <li>Описать также в анализе какие необоходимо внести изменения  в получившуюся у вас нейронную сеть если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET.
    </li>
</ol>

In [2]:
from __future__ import print_function
import keras # расскоментируйте эту строку, чтобы начать обучение
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [3]:
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [5]:
# установка параметров нейросети
batch_size = 320 #32
num_classes = 10
epochs = 30 #1
data_augmentation = False #True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [6]:
# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [7]:
# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [8]:
%%time
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(256, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# инициализация RMSprop optimizer
#opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='Adam', #opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    # запуск data augmentation через fit
    #datagen.fit(x_train)

    # запуск data augmentation через fit_generator
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)


Не используется data augmentation
Epoch 1/30
157/157 [==============================] - 24s 154ms/step - loss: 1.9186 - accuracy: 0.2809 - val_loss: 1.5531 - val_accuracy: 0.4323
Epoch 2/30
157/157 [==============================] - 12s 74ms/step - loss: 1.4265 - accuracy: 0.4777 - val_loss: 1.2208 - val_accuracy: 0.5596
Epoch 3/30
157/157 [==============================] - 12s 74ms/step - loss: 1.1819 - accuracy: 0.5771 - val_loss: 1.0549 - val_accuracy: 0.6175
Epoch 4/30
157/157 [==============================] - 12s 74ms/step - loss: 1.0381 - accuracy: 0.6326 - val_loss: 0.9339 - val_accuracy: 0.6696
Epoch 5/30
157/157 [==============================] - 12s 74ms/step - loss: 0.9252 - accuracy: 0.6727 - val_loss: 0.8685 - val_accuracy: 0.6956
Epoch 6/30
157/157 [==============================] - 12s 74ms/step - loss: 0.8622 - accuracy: 0.6941 - val_loss: 0.8007 - val_accuracy: 0.7239
Epoch 7/30
157/157 [==============================] - 12s 74ms/step - loss: 0.7814 - accuracy: 0.7254

In [9]:
# сохранение модели и весов
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('сохранить обученную модель как %s ' % model_path)

In [10]:
# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 9s 29ms/step - loss: 0.5541 - accuracy: 0.8288
Test loss: 0.5540534853935242
Test accuracy: 0.8288000226020813


### Выводы:
1. Если просто увеличить количестов эпох, то результаты улучшаются, но до определённого момента, после которого точность уже не растёт - выходит на плато.<br>
Если же увеличить batch_size, то вырастает скорость (расчёт вёлся на GPU), но зато падает точность. Для компенсации падения точности можно увеличить число эпох и тогда соотношение качество/ед.веремени будет больше. Т.е. за единицу времени потраченного на обучение получаем большее качество модели при большом batch_size и большом числе эпох.<br>
augmentation работает непропорционально долго. Теоретичестки с augmentation можно получить большее качество, но не всегда потраченное на неё время этого стоит.<br>
Изменил метод оптимизации на Adam - качество улучшилось.<br>
Добавил ещё один слой и увеличил число нейронов во внутренних слоях по обратной пирамиде (256, 128, 64) - качество ещё подросло.
2. Для MNIST, CIFAR100 и IMAGENET необходимо поменять число входных и выходных нейронов, т.к. размеры исходных картинок и количество искомых категорий в этих датасетах различное.